# On trigger signal acquisition

## Description

This example shows how to acquire 16k samples of signal on fast analog inputs.
Signal will be acquired when the internal trigger condition is meet.
Time length of the acquired signal depends on the time scale of a buffer
that can be set with a decimation factor.

TODO: describe some calculations

## Required hardware

- Red Pitaya
- Signal (function) generator

![wiring diagram](img/on_given_trigger_acquire_signal_on_fast_analog_input.png)

The `rp` object is an instance of the `redpitaya` class.
When the object is initialized, the FPGA bitstream is loaded and
memory mapped FPGA registers are mapped into software.
Repeating FPGA bitstream loading will cause all registers to reset,
while mapping the memory space multiple times will cause segmentation faults.
So untill this issues are handled by the driver
a `redpitaya` instance should be created only once.

In [ ]:
from redpitaya import redpitaya
rp = redpitaya()

In [ ]:
# generator configuration
rp.GenReset()
rp.GenFreq(0, 100000.0)
rp.GenAmp(0, 1.0)
rp.GenWaveform(0, rp.WAVEFORM_SINE)
rp.GenOutEnable(0)

In [ ]:
# acquisition configuration
size = rp.AcqGetBufSize()
rp.AcqReset()
rp.AcqSetDecimationFactor(1)
rp.AcqSetTriggerLevel(0, 0.1)
rp.AcqSetPostTriggerDelay(size//2) # place trigger in the middle of the buffer

size = 2048
channels = (0, 1)

# start acquisition process
rp.AcqStart()
# set trigger source to start acquisition
rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)
# wait in a loop for trigger state to chage from TRIG_STATE_WAITING
while rp.AcqGetTriggerSrc():
    pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = [rp.AcqGetOldestDataV(ch, size) for ch in channels];

In [ ]:
import time
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
output_notebook()

In [ ]:
N = size
x = np.arange(N) / rp.FS

colors = ('red', 'blue')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=500, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.yaxis.axis_label='voltage [V]'
p.y_range=Range1d(-1.2, 1.2)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [ ]:
while True:
    rp.AcqStart()
    rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)
    while rp.AcqGetTriggerSrc(): pass
    buff = [rp.AcqGetOldestDataV(ch, size) for ch in channels];
#    buff = np.absolute(np.fft.fft(buff))
    for i in channels:
        r[i].data_source.data['y'] = buff[i]
    rp.AcqStart()
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
#    time.sleep(0.05)